In [52]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import scipy.stats as stat

In [237]:
single_fits = pd.read_csv('model_results/single_fit_signed_initerror_learnwashout_results.csv').drop('Unnamed: 0', axis = 1).reset_index().drop('index', axis = 1)
dual_fits = pd.read_csv('model_results/dual_fit_signed_initerror_learnwashout_results.csv').drop('Unnamed: 0', axis = 1).reset_index().drop('index', axis = 1)


In [238]:
single_fits['AIC'] = 2*single_fits['gof'] + 2*2
dual_fits['AIC'] = 2*dual_fits['gof'] + 2*4

single_fits['BIC'] = 2*single_fits['gof'] + np.log(64*7)*2
dual_fits['BIC'] = 2*dual_fits['gof'] + np.log(64*7)*4


In [239]:
single_fits.loc[single_fits['p_id']%2 == 0, 'Emphasis'] = 'Speed'
single_fits.loc[single_fits['p_id']%2 == 1, 'Emphasis'] = 'Accuracy'

single_fits.loc[single_fits['p_id']%4 == 0, 'Rotation'] = 'Sudden'
single_fits.loc[single_fits['p_id']%4 == 1, 'Rotation'] = 'Sudden'
single_fits.loc[single_fits['p_id']%4 == 2, 'Rotation'] = 'Gradual'
single_fits.loc[single_fits['p_id']%4 == 3, 'Rotation'] = 'Gradual'


dual_fits.loc[dual_fits['p_id']%2 == 0, 'Emphasis'] = 'Speed'
dual_fits.loc[dual_fits['p_id']%2 == 1, 'Emphasis'] = 'Accuracy'

dual_fits.loc[dual_fits['p_id']%4 == 0, 'Rotation'] = 'Sudden'
dual_fits.loc[dual_fits['p_id']%4 == 1, 'Rotation'] = 'Sudden'
dual_fits.loc[dual_fits['p_id']%4 == 2, 'Rotation'] = 'Gradual'
dual_fits.loc[dual_fits['p_id']%4 == 3, 'Rotation'] = 'Gradual'


single_fits['Condition'] = single_fits['Rotation'] + ' ' + single_fits['Emphasis']

In [240]:
single_fits['Condition'].value_counts()

Sudden Speed        15
Sudden Accuracy     15
Gradual Speed       15
Gradual Accuracy    15
Name: Condition, dtype: int64

In [241]:
single_fits.loc[single_fits['AIC'] > dual_fits['AIC'], 'Condition'].value_counts()

Gradual Accuracy    12
Gradual Speed        9
Sudden Accuracy      6
Sudden Speed         3
Name: Condition, dtype: int64

In [242]:
stat.chisquare([12, 12, 8, 6])

Power_divergenceResult(statistic=2.8421052631578947, pvalue=0.41661546187232257)

In [243]:
single_fits.loc[single_fits['BIC'] > dual_fits['BIC'], 'Condition'].value_counts()

Gradual Accuracy    9
Sudden Accuracy     5
Gradual Speed       5
Sudden Speed        3
Name: Condition, dtype: int64

In [244]:
single_fits['LR_ratio'] = 2* (single_fits['gof'].values - dual_fits['gof'].values)

In [245]:
single_fits['chi2'] = stat.chi2.sf(single_fits['LR_ratio'], 2)

In [246]:
single_fits.loc[single_fits['chi2'] < 0.05, 'Condition'].value_counts()

Gradual Accuracy    12
Gradual Speed        9
Sudden Accuracy      5
Sudden Speed         3
Name: Condition, dtype: int64

In [247]:
9/15

0.6

# Cross Validation Comparisons

In [252]:
single_fits = pd.read_csv('model_results/single_fit_signed_initerror_learnwashout_results_cv.csv')
dual_fits = pd.read_csv('model_results/dual_fit_signed_initerror_learnwashout_results_cv.csv')

single_fits.loc[single_fits['p_id']%2 == 0, 'Emphasis'] = 'Speed'
single_fits.loc[single_fits['p_id']%2 == 1, 'Emphasis'] = 'Accuracy'

single_fits.loc[single_fits['p_id']%4 == 0, 'Rotation'] = 'Sudden'
single_fits.loc[single_fits['p_id']%4 == 1, 'Rotation'] = 'Sudden'
single_fits.loc[single_fits['p_id']%4 == 2, 'Rotation'] = 'Gradual'
single_fits.loc[single_fits['p_id']%4 == 3, 'Rotation'] = 'Gradual'


dual_fits.loc[dual_fits['p_id']%2 == 0, 'Emphasis'] = 'Speed'
dual_fits.loc[dual_fits['p_id']%2 == 1, 'Emphasis'] = 'Accuracy'

dual_fits.loc[dual_fits['p_id']%4 == 0, 'Rotation'] = 'Sudden'
dual_fits.loc[dual_fits['p_id']%4 == 1, 'Rotation'] = 'Sudden'
dual_fits.loc[dual_fits['p_id']%4 == 2, 'Rotation'] = 'Gradual'
dual_fits.loc[dual_fits['p_id']%4 == 3, 'Rotation'] = 'Gradual'


single_fits['Condition'] = single_fits['Rotation'] + ' ' + single_fits['Emphasis']

In [253]:
dual_fits['dual win'] = (single_fits['test gof'].values > dual_fits['test gof'].values).astype(int)

In [254]:
def compute_wins(dual_fits):
    # Drop column: 'Unnamed: 0'
    # dual_fits = dual_fits.drop(columns=['Unnamed: 0'])
    # Performed 1 aggregation grouped on columns: 'p_id', 'Rotation' and 2 other columns
    dual_fits = dual_fits.groupby(['p_id', 'Rotation', 'Emphasis', 'dual win']).agg(dualwin_count=('dual win', 'count')).reset_index()
    # Filter rows based on column: 'dual win'
    dual_fits = dual_fits[dual_fits['dual win'] == 1]
    return dual_fits

dual_fits_wins = compute_wins(dual_fits.copy())
dual_fits_wins['Condition'] = dual_fits_wins['Rotation'] + ' ' + dual_fits_wins['Emphasis']

In [257]:
dual_fits_wins.loc[dual_fits_wins['dualwin_count'] > 57, 'Condition'].value_counts()

Sudden Accuracy     3
Gradual Speed       3
Sudden Speed        1
Gradual Accuracy    1
Name: Condition, dtype: int64

In [232]:
2/15

0.13333333333333333

In [27]:
dual_fits_wins.Condition.value_counts()

Gradual Accuracy    16
Sudden Speed        16
Gradual Speed       15
Sudden Accuracy     15
Name: Condition, dtype: int64

In [17]:
2/15

0.13333333333333333